# Academy: Practicum Visualizing Galactic Chemical Evolution, Birth Radius, and Exoplanets

## 1. Goal of the practicum
The goal of this practicum is to learn how to visually connect:
- Galactic Chemical Evolution (GCE),
- stellar birth radius (r₍birth₎),
- stellar migration,
and the observed distribution of exoplanets.

By the end of the practicum, we will have a set of plots that:
- explain both scientific papers,
- can be interpreted intuitively, without equations,
- can be directly reused in the Academy, Celestial Chronicles, and Astro Minute formats.

## 2. Practicum 1 Metallicity gradient: [Fe/H] vs Galactocentric radius
### 2.1 What we want to show
- That the inner Galaxy is richer in metals.
- That this gradient evolved over time.
### 2.2 Axes and data
- X-axis: Galactocentric radius R (kpc)
- Y-axis: metallicity [Fe/H]
- Several curves or point clouds for different epochs:
- 10 Gyr
- 6–8 Gyr
- < 4 Gyr
### 2.3 Interpretation
- In early epochs, planets were possible only close to the Galactic center.
- Over time, the “planet-forming zone” expanded outward.

📌 Narrative:
*The Galaxy first learned how to make metals — and only then how to make planets.*


## 3. Practicum 2 Evolution of planetary building blocks
### 3.1 What we visualize
Individual elements or groups:
- Fe — planetary cores and gas giants
- Mg + Si — silicate planets
- O + C — ices and volatile components
### 3.2 Plot format
- X-axis: R (kpc)
- Y-axis: elemental abundances
- Multiple epochs
### 3.3 Why this matters
- To show that different types of planets “switch on” at different times.
- To explain why gas giants are more sensitive to Galactocentric radius than super-Earths.

📌 Narrative:
*Not all planets are possible everywhere and at all times.*


## 4. Practicum 3 Birth radius: where planetary systems were actually born
### 4.1 What we compare
Birth radius $r_{birth}$ for stars:
- with planets,
- without planets.
### 4.2 Plot type
KDEs or histograms:
- X-axis: $r_{birth}$
- Y-axis: density
### 4.3 Expected result
Planet-hosting systems are shifted toward smaller $r_{birth}$.
The effect is especially strong for massive planets.

📌 Narrative:
*We observe planets far from where they were born.*


## 5. Practicum 4 Planet formation efficiency vs r₍birth₎
### 5.1 Definition
Planet formation efficiency = the relative fraction of stars hosting planets at a given r₍birth₎.
### 5.2 Plot
X-axis: $r_{birth}$
Y-axis: relative frequency
Two curves:
massive planets,
low-mass planets.
### 5.3 Interpretation
A steep decline in efficiency with increasing $r_{birth}$ for gas giants.
A much flatter dependence for small planets.

📌 Narrative:
*Giants are the children of the Galaxy’s wealthy neighborhoods.*


## 6. Practicum 5 Stellar migration: a schematic, not precise dynamics
### 6.1 Important note
This is not a numerical orbit calculation, but a conceptual visualization.
### 6.2 What we draw
Radius axis $R$
Arrows:
$r_{birth}$ → $R_{now}$
Several representative trajectories:
- inner → outer disk,   
- moderate migration,
- almost no migration.
### 6.3 Why this is needed
To show why chemistry and present-day position do not match.
To link $r_{birth}$ with observations.

📌 Narrative:
*The Galaxy is a system of currents.*


## 7. Minimal Python stack (if we move to code)
If we decide to actually generate the plots:
- numpy,
- pandas,
- matplotlib (no 3D),
- scipy (KDE).

Data sources:
- synthetic data (for conceptual clarity), or
- real catalogs: SWEET-Cat / HARPS.

👉 At the first stage, synthetic data are sufficient to convey the idea.


## 8. What immediately feeds into content
Academy: all plots with full explanations.

Celestial Chronicles: 2–3 plots plus schematic diagrams.

Astro Minute: 1 plot + 1 key sentence.

Stellar Attractor Universe: the logic of migrating worlds.


## 9. Final takeaway of the practicum
If we combine:
- GCE,
- birth radius,
- stellar migration,
- and these visualizations, 

then exoplanets stop being a random collection of discoveries, and become a natural outcome of the Milky Way’s evolution.
